In [13]:
from classes.data_loader import DataLoader
from classes.quantile_model import LassoQuantileModel
from sklearn.metrics import r2_score

In [14]:
data = DataLoader("data/usa.csv")

/Users/shoutianze/Desktop/F23/36490/36490-F23-Group1/classes/data_loader.py:17: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data: DataFrame = pd.read_csv(csv_file_path)


In [15]:
best_model, best_r2, best_lambda = LassoQuantileModel.validate(data, 19800101, 19950101, 19950101, 20000101, [0, 0.0000001, 0.00001, 0.001])
best_r2, best_lambda

(-9.564906779229954, 0.001)

In [10]:
r2_score(data.get_y_quantiles(data.slice(20000101, 20010101)), best_model.predict(20000101, 20010101))

-0.17552299445653174

In [6]:
data.get_y_quantiles(data.slice(20000101, 20010101))

array([0.19097305, 0.44310458, 0.80138028, ..., 0.60767947, 0.55999583,
       0.73098021])

In [8]:
best_model.predict(20000101, 20010101)

array([0.44756141, 0.50342833, 0.40124911, ..., 0.63957787, 0.77478834,
       0.3178959 ])